Code from https://www.analyticsvidhya.com/blog/2018/11/introduction-text-summarization-textrank-python/

In [3]:
import numpy as np
import pandas as pd
import nltk
nltk.download('punkt')
import re

[nltk_data] Downloading package punkt to
[nltk_data]     /home/tomasmizera/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [12]:
df = pd.read_csv("../data/tennis_articles.csv", encoding='utf-8')

In [13]:
df

,article_id,article_title,article_text,source
0,1,"I do not have friends in�tennis, says Maria Sh...",Maria Sharapova has basically no friends as te...,https://www.tennisworldusa.org/tennis/news/Mar...
1,2,Federer defeats Medvedev to advance to 14th Sw...,"BASEL, Switzerland (AP) � Roger Federer advanc...",http://www.tennis.com/pro-game/2018/10/copil-s...
2,3,Tennis: Roger Federer ignored deadline set by ...,Roger Federer has revealed that organisers of ...,https://scroll.in/field/899938/tennis-roger-fe...
3,4,Nishikori to face off against Anderson in Vien...,Kei Nishikori will try to end his long losing ...,http://www.tennis.com/pro-game/2018/10/nishiko...
4,5,Roger Federer has made this huge change to ten...,"Federer, 37, first broke through on tour over ...",https://www.express.co.uk/sport/tennis/1036101...
5,6,Rafael Nadal: World No 1 ARRIVES for Paris Mas...,Nadal has not played tennis since he was force...,https://www.express.co.uk/sport/tennis/1037119...
6,7,"TENNIS.COM PODCAST: POINT DEFENSE, RANKING DRO...","Tennis giveth, and tennis taketh away. The end...",http://www.tennis.com/pro-game/2018/10/tennisc...
7,8,Tennis journalist�s heartbreaking insight on T...,I PLAYED golf last week with Todd Reid. He pic...,https://www.foxsports.com.au/tennis/tennis-jou...


In [17]:
df['article_text']

0    Maria Sharapova has basically no friends as te...
1    BASEL, Switzerland (AP) � Roger Federer advanc...
2    Roger Federer has revealed that organisers of ...
3    Kei Nishikori will try to end his long losing ...
4    Federer, 37, first broke through on tour over ...
5    Nadal has not played tennis since he was force...
6    Tennis giveth, and tennis taketh away. The end...
7    I PLAYED golf last week with Todd Reid. He pic...
Name: article_text, dtype: object

In [18]:
from nltk.tokenize import sent_tokenize

sentences = []
for s in df['article_text']:
    sentences.append(sent_tokenize(s))
    
sentences = [ y for x in sentences for y in x ]

In [23]:
sentences[16:18]

['But ultimately tennis is just a very small part of what we do.',
 "There are so many other things that we're interested in, that we do.'"]

In [24]:
clean_sentences = pd.Series(sentences).str.replace("[^a-zA-Z]", " ")
clean_sentences = [s.lower() for s in clean_sentences]

In [25]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     /home/tomasmizera/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [26]:
from nltk.corpus import stopwords
stop_words = stopwords.words('english')

In [27]:
def remove_stopwords(sen):
    sen_new = " ".join([i for i in sen if i not in stop_words])
    return sen_new

clean_sentences = [remove_stopwords(r.split()) for r in clean_sentences]

---

#### Vectorizer

In [62]:
# from sklearn.feature_extraction.text import CountVectorizer

In [63]:
# vectorizer = CountVectorizer()
# vectorizer.fit([' '.join(clean_sentences)])

In [64]:
# sentence_vectors = []
# for i in clean_sentences:
#     if len(i) != 0:
#         v = sum([vectorizer.vocabulary_.get(w, np.zeros((100,))) for w in i.split()])/(len(i.split())+0.001)
#         v = np.array(v)
#     else:
#         v = np.zeros((100,))
#     sentence_vectors.append(v)

In [65]:
# sentence_vectors[:5]

### Vectorizer v2 - Glove

In [66]:
word_embeddings = {}
f = open('../data/glove/glove.6B.100d.txt', encoding='utf-8')

In [67]:
for line in f:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    word_embeddings[word] = coefs
f.close()

In [68]:
len(word_embeddings)

400000

In [69]:
sentence_vectors = []
for i in clean_sentences:
    if len(i) != 0:
        v = sum([word_embeddings.get(w, np.zeros((100,))) for w in i.split()])/(len(i.split())+0.001)
    else:
        v = np.zeros((100,))
    sentence_vectors.append(v)

In [77]:
sentence_vectors[:1][0][:5]

array([ 0.05148256,  0.11054468,  0.6949994 ,  0.1891681 , -0.09580777],
      dtype=float32)

---

---

In [78]:
# similarity matrix
sim_mat = np.zeros([len(sentences), len(sentences)])

In [79]:
from sklearn.metrics.pairwise import cosine_similarity

In [80]:
for i in range(len(sentences)):
    for j in range(len(sentences)):
        if i != j:
            sim_mat[i][j] = cosine_similarity(sentence_vectors[i].reshape(1,100), sentence_vectors[j].reshape(1,100))[0,0]

---

### PageRank

In [81]:
import networkx as nx

In [82]:
nx_graph = nx.from_numpy_array(sim_mat)
scores = nx.pagerank(nx_graph)

---

### Summary Extraction

In [98]:
ranked_sentences = sorted(((scores[i],s) for i, s in enumerate(sentences)), reverse=True)

In [101]:
for i in range(10):
    print(ranked_sentences[i])

(0.008542794969989746, '�I was on a nice trajectorythen,� Reid recalled.�If I hadn�t got sick, I think I could have started pushing towards the second week at the slams and then who knows.� Duringa comeback attempt some five years later, Reid added Bernard Tomic and 2018 US Open Federer slayer John Millman to his list of career scalps.')
(0.7500584255627886, 'Major players feel that a big event in late November combined with one in January before the Australian Open will mean too much tennis and too little rest.')
(0.008452754148043057, "So I'm not the one to strike up a conversation about the weather and know that in the next few minutes I have to go and try to win a tennis match.")
(0.008446753121153963, 'Speaking at the Swiss Indoors tournament where he will play in Sunday�s final against Romanian qualifier Marius Copil, the world number three said that given the impossibly short time frame to make a decision, he opted out of any commitment.')
(0.008431508154861666, 'Currently in ni

In [100]:
ranked_sentences[1] = (ranked_sentences[1][0] + (0.89-0.23)/max(0.89, 0.23), ranked_sentences[1][1])

In [105]:
(0.89-0.23)/max(0.89, 0.23)

0.7415730337078652

Another option https://www.analyticsvidhya.com/blog/2020/12/tired-of-reading-long-articles-text-summarization-will-make-your-task-easier/